In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib_inline 
import matplotlib.pyplot as plt 

test_set=pd.read_csv('test.csv') 
train_set=pd.read_csv('train.csv') 
gender=pd.read_csv('gender_submission.csv') 

In [ ]:
print(test_set.head())
print(test_set.info())  

#print(train_set.head(40)) 
#print(train_set.info()) 

#print(gender.head(40)) 
#print(gender.info()) 

#train_set.hist() 
#plt.show()  

#test_set.hist() 
#plt.show() 

#gender.hist() 
#plt.show() 

#print(train_set.isnull().sum()) 
print(test_set.isnull().sum()) 

NameError: name 'test_set' is not defined

In [ ]:
train_corr=train_set.select_dtypes(include=[np.number]) 
corr_matrix=train_corr.corr() 
print(corr_matrix["Survived"].sort_values(ascending=False)) 

In [ ]:
train_set_labels=train_set["Survived"]

In [ ]:
from sklearn.impute import SimpleImputer 
imputer=SimpleImputer(strategy="median") 
train_num=train_set.drop(["Survived", "Sex", "Embarked", "Ticket", "Name", "Cabin"], axis=1) 
imputer.fit(train_num)  
print(imputer.statistics_)
x=imputer.transform(train_num)  
train_num_tr=pd.DataFrame(x, columns=train_num.columns, index=train_num.index) 
print(train_num_tr) 


In [ ]:
from sklearn.preprocessing import OrdinalEncoder 
oe=OrdinalEncoder() 
train_set["Embarked"].fillna(train_set["Embarked"].mode()[0], inplace=True)
train_cat=train_set[["Sex", "Embarked"]] 
train_cat_en=oe.fit_transform(train_cat) 
print(train_cat_en[:40]) 
train_cat_tr=pd.DataFrame(train_cat_en, columns=["Sex", "Embarked"], index=train_set.index) 
print(train_cat_tr) 

In [ ]:
train_set_prepared=pd.concat([train_num_tr, train_cat_tr], axis=1) 
print(train_set_prepared.head(10)) 

In [ ]:
from sklearn.model_selection import cross_val_predict
from lightgbm import LGBMClassifier 
lgbm=LGBMClassifier(n_estimators=100, learning_rate=0.01, max_depth=5, random_state=42) 
lgbm.fit(train_set_prepared, train_set_labels) 
lgbm_pred=cross_val_predict(lgbm, train_set_prepared, train_set_labels, cv=5) 
print("predictions= ", lgbm_pred[:20]) 
print("labels= ", list(train_set_labels[:20]))  

In [ ]:
from sklearn.metrics import accuracy_score 
acc=accuracy_score(train_set_labels, lgbm_pred)  
print(acc) 

In [ ]:
test_corr=test_set.select_dtypes(include=[np.number]) 
corr2_matrix=test_corr.corr() 
print(corr2_matrix["Survived"].sort_values(ascending=False)) 

In [ ]:
test_num=test_set.drop(["Sex", "Embarked", "Ticket", "Name", "Cabin"], axis=1) 
print(imputer.statistics_)
y=imputer.transform(test_num)  
test_num_tr=pd.DataFrame(y, columns=test_num.columns, index=test_num.index) 
print(test_num_tr) 

In [ ]:
test_set["Embarked"].fillna(test_set["Embarked"].mode()[0], inplace=True)
test_cat=test_set[["Sex", "Embarked"]] 
test_cat_en=oe.transform(test_cat) 
print(test_cat_en[:40]) 
test_cat_tr=pd.DataFrame(test_cat_en, columns=["Sex", "Embarked"], index=test_set.index) 
print(train_cat_tr) 

In [ ]:
test_set_prepared=pd.concat([test_num_tr, test_cat_tr], axis=1) 
print(test_set_prepared.head(10)) 

In [ ]:
test_pred=lgbm.predict(test_set_prepared)  

In [ ]:
final=pd.DataFrame( {"PassengerId": test_set["PassengerId"], "Survived": test_pred} ) 
final.to_csv("titanic_final_submission.csv", index=False) 